# First simulation with OpenMM

## 1. Force field definition and energy minimization

The first simulation below is a minimal example of a molecular system: just a single water molecule. Initially, only the energy of this system is minimized. This example consists of a few steps:

### Import OpenMM, SimTK and NumPy python packages

These packages must be imported before one can use OpenMM. In this example NumPy is used to specify the array with initial positions and to analyze simulation results.

In [4]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

In [5]:
%matplotlib inline
import numpy as np
import pandas

import matplotlib.pyplot as plt
import plotly.express as px

### Units

Internally, OpenMM uses internally consistent units based on the SI system, as documented [here](http://docs.openmm.org/latest/userguide/theory.html#units). One may easily convert values with units to a different unit, as shown in the examples below.

In [6]:
# Assign a value of 10 picoseconds
time = 10 * picosecond  # 10 * picoseconds would also work.
print("Unit of variable time:", time.unit)
print("time:", time)
print("time [s]:", time / second)
print("time [s]:", time / seconds)
print("time [fs]:", time.value_in_unit(femtosecond))

# Simtk also knows a few important constants.
print("Boltzmann's constant:", BOLTZMANN_CONSTANT_kB)
print("Avogadro's constant:", AVOGADRO_CONSTANT_NA)

Unit of variable time: picosecond
time: 10 ps
time [s]: 1e-11
time [s]: 1e-11
time [fs]: 10000.0
Boltzmann's constant: 1.3806504e-23 J/K
Avogadro's constant: 6.02214179e+23 /mol


A complete overview of units and constant can be found the Python wrapper source code for OpenMM: 

* Units: https://github.com/openmm/openmm/blob/master/wrappers/python/simtk/unit/unit_definitions.py
* Constants: https://github.com/openmm/openmm/blob/master/wrappers/python/simtk/unit/constants.py

### Define the topology

Normally, the topology is defined by loading a PDB file. In this example, it is defined from scratch by creating objects in Python code, to show explicitly how the topology is built up. Similar data structures are created when load topologies from files.

Note that a topology in biomolecular simulations consists of a group of chains, where a chain consists of multiple residues and every residue contains of a set of atoms, which are connected by bonds. This terminology was originally defined for describing proteins in the PDB file format, see http://www.wwpdb.org/documentation/file-format. In biomolecular simulations, the same terminology is nowadays also used for other things than proteins. For example, here we have a chain with a single residue, a water molecule, containing three atoms and two bonds.

In [8]:
topology = Topology()
chain    = topology.addChain()

residue = topology.addResidue("water", chain)

element_O = Element.getByAtomicNumber(8)
element_H = Element.getByAtomicNumber(1)

atom0 = topology.addAtom("O", element_O, residue)
atom1 = topology.addAtom("H", element_H, residue)
atom2 = topology.addAtom("H", element_H, residue)

topology.addBond(atom0, atom1)
topology.addBond(atom0, atom2)

### Set up the simulation

This part consists of a few steps:

a. Select the TIP3-FB **force field definition** for water, see https://doi.org/10.1021/jz500737m. The file `tip3pfb.xml` is included in the OpenMM installation. It's contents can also be viewed online, see https://github.com/openmm/openmm/blob/master/wrappers/python/simtk/openmm/app/data/amber14/tip3pfb.xml

b. Define a **system**, which is essentially an object implementing the selected force field for our topology. This object can compute energies and forces.

c. Define an **integrator**, which implements a molecular dynamics algorithm, in this case the Verlet algorithm sampling the NVE ensemble. Even though we will initially only carry out a geometry optimization, OpenMM requires us to define an integrator.

d. Define a **simulation**, which has the following responsibilities:

   - keep track of the current state (atomic positions etc.), 
   
   - decide which implementation to use (CPU versus GPU)
   
   - drive the integrator and 
   
   - write output files (reporters).
  
This simple example does not write any output yet. The initial positions are provided as a NumPy array, multiplied by a unit. Each row in the array contains the X, Y and Z coordinates of one atom, in the same order as the atoms in the topology. Normally, also these initial positions are loaded from a PDB file.

In [9]:
# a. Select the TIP3P-FB force field.
forcefield = ForceField('amber14/tip3pfb.xml')

# b. Create an object to compute energies and forces for our topology.
system = forcefield.createSystem(topology, nonbondedCutoff=1*nanometer)

# c. Definition an integrator, mandatory.
integrator = VerletIntegrator(1 * femtoseconds)

# d. A simulation object in OpenMM combines topology, system and integrator.
simulation = Simulation(topology, system, integrator)

simulation.context.setPositions(np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 1.0, 0.0],
]) * angstroms)

01_water.ipynb          ljscalars.csv           traj.pdb
02_lennard_jones.ipynb  ljtraj.dcd              water.pdb
ljinit.pdb              scalars.csv


### Minimize the energy

Also this part is split up in a few steps:

a. Before minimizing, compute and print the **energy** of the **initial** state.

b. **Minimize** the energy. This is a black-box procedure, using the L-BFGS algorithm. Tracking the progress during the minimization is not possible yet, see https://github.com/openmm/openmm/issues/1155.

c. Compute and print the **energy** of the **final** state.

In [10]:
# a. Print the energy before minimizing.
state0 = simulation.context.getState(getEnergy=True)
print(state0.getPotentialEnergy())

# b. Minimize the energy.
simulation.minimizeEnergy()

# c. Print the energy afterwards.
state1 = simulation.context.getState(getEnergy=True)
print(state1.getPotentialEnergy())

42.62383270263672 kJ/mol
1.2469004104787018e-05 kJ/mol


## 1. Inspection of OpenMM data structures

Before performing other simulations, we take a closer look at the objects created in the previous example to understand how OpenMM works. To access the internals, we use the so-called Python Application Programming Interface (API) of OpenMM, which is documented here: http://docs.openmm.org/latest/api-python/index.html.

The API documentation contains tables with class names, like `Topology`, with there corresponding purpose. Clicking on the class name brings you to a page with more details.

### Topology

The `Topology` class is documented here: http://docs.openmm.org/latest/api-python/generated/simtk.openmm.app.topology.Topology.html#simtk.openmm.app.topology.Topology

In [11]:
# The print lines below show details of the topology.
print(topology)
print("Number of atoms:", topology.getNumAtoms())
print("Periodic boundaries:", topology.getUnitCellDimensions())

for iatom, atom in enumerate(topology.atoms()):
    
    # try  print(vars(atom)) to learn more 
    print("Name and mass of atom {}: {}, {}".format(iatom, atom.name, atom.element.mass))

<Topology; 1 chains, 1 residues, 3 atoms, 2 bonds>
Number of atoms: 3
Periodic boundaries: None
Name and mass of atom 0: O, 15.99943 Da
Name and mass of atom 1: H, 1.007947 Da
Name and mass of atom 2: H, 1.007947 Da


**<mark> Exercsie </mark>**

> Extend the code cell above to loop over the bonds and to print the corresponding pair of atoms.

### System

The relevant API documentation can be found here:

* `System` class: http://docs.openmm.org/latest/api-python/generated/simtk.openmm.openmm.System.html#simtk.openmm.openmm.System
* `HarmonicBondForce` class: http://docs.openmm.org/latest/api-python/generated/simtk.openmm.openmm.HarmonicBondForce.html#simtk.openmm.openmm.HarmonicBondForce

In [14]:
# Loop over all contributions to the force evaluation.
# Note that not all terms are real force-field contributions.
# The last one zeros to center-of-mass momentum at every step.

for force in system.getForces():
    print(force)
    
print()

<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x111dba570> >
<simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x111dba540> >
<simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x111dba300> >
<simtk.openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x111dba2a0> >



In [15]:
# The force field contains two harmonic bond terms, with the same parameters
force_bond = system.getForce(0)

for ibond in range(force_bond.getNumBonds()):
    print(force_bond.getBondParameters(ibond))
    print()

[0, 1, Quantity(value=0.101181082494, unit=nanometer), Quantity(value=462750.4, unit=kilojoule/(nanometer**2*mole))]

[0, 2, Quantity(value=0.101181082494, unit=nanometer), Quantity(value=462750.4, unit=kilojoule/(nanometer**2*mole))]



**<mark> Exercsie </mark>**

> Add code to show the angle force terms of the system and the corresponding parameters. Do these match the definition of the force field in the `tip3pfb.xml` file?

### System state

A `Simulation` object has a `context` attribute, through which many details of the simulation can be accessed. Here we use it to retrieve the current state of our molecular model. The `Context` class is documented here: http://docs.openmm.org/latest/api-python/generated/simtk.openmm.openmm.Context.html#simtk.openmm.openmm.Context

Below, the `getState` method retrieves a `State` object with results for the final geometry of the water molecule. The `State` class is documented here: http://docs.openmm.org/latest/api-python/generated/simtk.openmm.openmm.State.html#simtk.openmm.openmm.State

In [16]:
# Simply extract and print information
state = simulation.context.getState(
    getPositions=True, getForces=True, getEnergy=True)

print("Potential energy: ", state.getPotentialEnergy())
print()
print("Atomic positions")
print(state.getPositions(asNumpy=True))
print()
print("Forces")
print(state.getForces(asNumpy=True))
print()

Potential energy:  1.2469004104787018e-05 kJ/mol

Atomic positions
[[ 0.          0.00534427  0.00534427]
 [ 0.         -0.01060477  0.10526051]
 [ 0.          0.1052605  -0.01060477]] nm

Forces
[[ 0.          1.6560204   1.66786516]
 [ 0.         -1.40375566 -0.26247871]
 [ 0.         -0.25226474 -1.40538645]] kJ/(nm mol)



In [18]:
# One can use NumPy to get a few geometry properties:

# The length of the first O-H bond, in nanometers.
pos = state.getPositions(asNumpy=True)
print(np.linalg.norm(pos[1] - pos[0]))

# The H-O-H angle, in degrees
d01 = pos[1] - pos[0]
d02 = pos[2] - pos[0]
d01 /= np.linalg.norm(d01)
d02 /= np.linalg.norm(d02)
cosine = np.dot(d01, d02)
print(np.arccos(cosine)*180/np.pi)

0.10118115878989922
108.13854873886349


## 3. PDB output and visualization

The following is a minimal example to write out the current positions to a PDB file, after which NGLView is used to visualize the structure. See http://nglviewer.org/nglview/latest/.

In [27]:
# NGLView is a structure and trajectory visualization that integrates well with Jupyter Notebooks.
import nglview

# Write out a PDB file
with open('water.pdb', 'w') as outfile:
    
    PDBFile.writeFile(topology, pos, outfile)

# Visualize
nglview.show_file('water.pdb')

NGLWidget()

## 4. A short molecular dynamics simulation

In the code cells above, we have defined a verlet integrator, which we will use in this section.

Before performing the MD, some more initialization steps are needed:

- Velocities are assigned random values sampled from the Maxwell-Boltzmann distribution.

- Three reporters are attached to the simulation to store the output in files and to track the progress on screen. The reporter API is documented here: http://docs.openmm.org/latest/api-python/app.html#reporting-output

In [28]:
# Initialize velocities with random values at 300K. # The random seed is fixed for reproducability

simulation.context.setVelocitiesToTemperature(300, 13453454)

# Remove existing reporters, in case this cell is executed more than once.

simulation.reporters = []

# Write a frame to the PDB trajectory every step.

simulation.reporters.append(PDBReporter('traj.pdb', 1))

# Write scalar properties to a CSV file every step.

simulation.reporters.append(StateDataReporter(
    "scalars.csv",
    1,
    time=True,
    potentialEnergy=True,
    totalEnergy=True,
    temperature=True))

# Write scalar properties to screen every 100 steps.
from sys import stdout
simulation.reporters.append(StateDataReporter(
    stdout,
    100,
    step=True,
    totalEnergy=True,
    temperature=True))

# Actually run the molecular dynamics simulation.
simulation.step(1000)

#"Step","Total Energy (kJ/mole)","Temperature (K)"
100,4.267331190407276,156.78038471562368
200,4.242004364728928,166.66902809263848
300,4.25336766242981,164.42550623197354
400,4.255927503108978,161.56585499548768
500,4.284445489291102,153.08941228087656
600,4.287395387887955,154.806316779006
700,4.289990524761379,152.8761404226752
800,4.314642697572708,143.20974651580644
900,4.327713787555695,139.28074830486207
1000,4.345395520329475,138.31472678481842


We will load the trajectory again with MDTraj and pass on this information to nglview to visualize the motion of the water molecule. See http://mdtraj.org/.

In [29]:
import mdtraj
nglview.show_mdtraj(mdtraj.load('traj.pdb'))

NGLWidget(max_frame=999)

The following example shows how to load the data from the CSV file and plot it, using Pandas. Under the hood pandas uses MatPlotLib to make the figure. In the example below, two curves are put on the same plot by plotting them in the same MatPlotLib axes.

See https://pandas.pydata.org/ and https://matplotlib.org/.

Because the OpenMM write slightly non-standard CSV output, the header of the first column contains a few spurious characters.

In [32]:
df = pandas.read_csv("scalars.csv")
df

,"#""Time (ps)""",Potential Energy (kJ/mole),Total Energy (kJ/mole),Temperature (K)
0,0.001,0.536689,4.293145,150.599086
1,0.002,1.248634,4.377869,125.453328
2,0.003,0.938369,4.338937,136.331269
3,0.004,0.153118,4.244037,164.007930
4,0.005,0.234108,4.257834,161.314125
5,0.006,1.003022,4.356277,134.434445
6,0.007,1.091419,4.368658,131.386908
7,0.008,0.328320,4.273784,158.176558
8,0.009,0.038146,4.236131,168.300336
9,0.010,0.739783,4.320378,143.548694


In [37]:
px.line(x=df.iloc[:,0], y=df.iloc[:,3])

The oscillations in the potential energy are normal. Due to the molecular vibrations, there is a continuous exchange between kinetic and potential energy. In principle, the total energy should be conserved exactly. In simulations, it is only approximately conserved because the Velocity Verlet scheme uses a finite-difference approximation. For practical applications, it is sufficient to observe no systematic drift in the conserved quantity.

**<mark> Exercsie </mark>**

> Experiment with this notebook to get some experience with molecular dynamics simulations:
>
> - The timestep is set to $1 \text{ fs}$ when creating the `VerletIntegrator` object. Increase the time step until the total energy is no longer conserved. What happens?
>
> - Replace the `VerletIntegrator` by a `LangevinIntegrator` at a temperature of 300 Kelvin, with a friction coefficient of 10 inverse picoseconds and step size of 1 femtosecond. The API documentation can be found here: http://docs.openmm.org/latest/api-python/generated/simtk.openmm.openmm.LangevinIntegrator.html#simtk.openmm.openmm.LangevinIntegrator
>
> - Make modifications to the topology to understand which parts are essential to generate the system object with the correct parameters. Try making changes to: (i) the atom names, (ii) the element numbers, (iii) the bonds, (iv) the name of the residue and (v) the order of the atoms.